<img src=images/logo.png align='right' width=200>

# Time Series Aggregations

## Goal

Our next step is to get introduced to a real time series dataset and learn some fundamental analysis techniques.

We shall first focus on how to group the data by time periods.

## Program
- [Reading in Time Series Data](#read)
- [Aggregations: `resample()`](#agg1)
- [Aggregations: `transform()`](#agg2)
- [Summary](#sum)


In [ ]:
import pandas as pd

<a id='read'></a>

## Reading in Time Series Data
<img src=images/plug.jpeg  width=200>

Throughout the this notebook we will use the *household power consumption* dataset. It comes from [UCI ML repo](https://archive.ics.uci.edu/ml/datasets/individual+household+electric+power+consumption) and contains detailed power consumption time series data of a single household in Paris between 2006 and 2010.

In [ ]:
power_raw = pd.read_csv('data/household_power_consumption.csv')
power_raw.head()

Notice how our time data is currently recognised as strings.

In [ ]:
power_raw.info()

Typically timestamp data is contained in a separate column of standard strings. However, we want to make it machine-readable. In order to achieve this, we can set `parse_dates` with the list of columns to be converted to Pandas timestamp when reading the data with `pd.read_csv`. This automatically identifies the format of the dates. 

In [ ]:
power_parsed = pd.read_csv('data/household_power_consumption.csv', parse_dates=['ts'])
power_parsed.head()

In [ ]:
power_parsed.info()

Explicit formatting may be a good idea if we suspect possible errors and non-standard formats in some of the rows. This will ensure that we get an error message if any value does not match our specified format.

In [ ]:
(
    power_raw
    .assign(ts=pd.to_datetime(power_raw['ts'], format='%Y-%m-%d %H:%M:%S'))
)

We can then access information in the dataset using standard Pandas techniques. 

Below we demonstrate how to select all the data recorded on 17th December 2006.

In [ ]:
(
    power_parsed
    .loc[power_parsed(power_parsed['ts'] >= pd.Timestamp('2006-12-17')) &
                     (power_parsed['ts'] < pd.Timestamp('2006-12-18'))]
)

In Pandas, we can also perform selections with `lambda` functions. They enable us to call the current version of the DataFrame (as opposed to having to refer to named variables) and allow for greater code reusability.

In [ ]:
(
    power_parsed
    .loc[lambda df: (df['ts'] >= pd.Timestamp('2006-12-17')) &
                    (df['ts'] < pd.Timestamp('2006-12-18'))]
)

However, for most time-series analysis functionality, we benefit from not only parsing the dates correctly, but also setting the dates as the index in the Pandas DataFrame. 

Let's illustrate the data without the datetime as index: 

In [ ]:
power = pd.read_csv('data/household_power_consumption.csv', 
                    parse_dates=['ts'], index_col='ts')
power.head()

Now it is much simpler to select the dates from a particular range.

In [ ]:
power.loc['2006-12-17']

In [ ]:
power.loc['2006-12-17':'2007-01-01']

We can even filter using different date formats!

In [ ]:
power.loc['17th December 2006']

It also provides us with an advantage with plotting:

In [ ]:
power.plot(figsize=(16,4));

Compare this to our *unformatted* data, which does not use the time data as the index:

In [ ]:
power_raw.plot(x='ts', y='consumption', figsize=(16,4))

Note how the x-axis is no longer nicely formatted? Pandas reads them as strings and will select x-ticks based on even intervals.

We may also want to do some histograms to see how our data is distributed. 

Let's look at the 2006 and 2007 - do you notice a difference?

In [ ]:
power.loc['2006'].plot(kind='hist')

In [ ]:
power.loc['2007'].plot(kind='hist')

<a id='agg1'></a>

## Aggregations: `resample()`

Another advantage of the datetime-index approach is that it provides us with some functionality for easy time-based aggregations. One such aggregation is `resample`. 

For example, we can easily calculate the _mean_ per _year_ by running:

In [ ]:
power.resample('Y').mean()

You can also run the same aggregation per month `M`, week `W`, day `D` or quarter `Q`. Custom aggregation periods are also possible, for example per 4 weeks `4W` or per 3 months `3M`. See [here] for a more comprehensive list of offsets, that can be as specific as _'Business Month Begin'_.

[here]: https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

If the index is a timestamp that also includes times, then you can also aggregate per hour. The script below demonstrates this by calculating the mean per 3 hours.

In [ ]:
power.resample('3H').mean().head()

In [ ]:
power.resample('3H').sum().head()

We can also use general `.agg()` methods here to apply multiple aggregators.

In [ ]:
(
    power
    .resample('M')
    .agg(monthly_mean = ('consumption', 'mean'),
         monthly_var = ('consumption', 'var'),
         monthly_spread = ('consumption', lambda month_df: month_df.max() - month_df.min())
        )
    .head()
)

<a id='agg2'></a>

## Aggregations: `transform()`

Notice how aggregating results in a smaller DataFrame than before.

In [ ]:
print(power.shape)

In [ ]:
print((
    power
    .resample('Y')
    .agg('mean')   
).shape)

Using `transform()` allows to keep the original range of indices (in seconds) after aggregation. Resulting aggregated statistics then are the same for each aggregation period (each hour in our case):

In [ ]:
(
    power
    .resample('Y')
    .transform('mean')   
)

In [ ]:
print((
    power
    .resample('Y')
    .transform('mean')   
).shape)

Using this with the `assign` method allows us to add new columns to the original dataset.

In [ ]:
(
    power
    .assign(monthly_mean = power.resample('M')['consumption'].transform('mean'),
           monthly_var = power.resample('M')['consumption'].transform('var'),
           monthly_spread = power.resample('M')['consumption'].transform(
               lambda month_df: month_df.max() - month_df.min())
           )
)

<a id='as'></a>
## <mark>Exercise: Resampling

Transform the dataset into consecutive five-day-periods:

- Show the total energy used in the five-day-periods

- Find the five-day-period in which the most energy is consumed in total




In [ ]:
# %load answers/power-resampling.py

<a id='sum'></a>
## Summary

We have covered: 
- How to properly read in time series data in Pandas, and why it is important to set the date as an index
- How to aggregate over time periods with Pandas
